# Figure 2
- Py Kernel with rpy2 integration for R and Python

In [16]:
import pandas as pd
import sys
sys.path.insert(1, '/Users/brinkley97/Documents/development/')
import utils

# Load Data

In [34]:
path_to_data =  "lab-kcad/datasets/tiles_dataset/" 
work_data = "figure_2/physical/diurnal_work_lm_6.csv.gz"
off_data = "figure_2/physical/diurnal_off_lm_6.csv.gz"

In [35]:
work_df = utils.load_gzip_csv_data(path_to_data, work_data)
off_df = utils.load_gzip_csv_data(path_to_data, off_data)

In [36]:
work_df

,Unnamed: 0,work,time,rest,step_ratio,run_ratio,id,age,gender,shift
0,work,work,3rd,0.870900,0.294175,0.236790,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
1,work,work,5th,0.628179,0.492380,0.402107,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
2,work,work,6th,0.901501,0.477633,0.377808,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
3,work,work,1st,0.948894,0.374328,0.288105,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
4,work,work,4th,0.764789,0.324393,0.271522,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
...,...,...,...,...,...,...,...,...,...,...
607,work,work,5th,0.572392,0.268371,0.162666,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day
608,work,work,6th,0.788603,0.169805,0.098312,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day
609,work,work,1st,0.986683,0.003109,0.001196,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day
610,work,work,4th,0.610316,0.271353,0.172747,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day


In [37]:
off_df

,Unnamed: 0,work,time,rest,step_ratio,run_ratio,id,age,gender,shift
0,off,off,3rd,0.749103,0.398709,0.315142,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
1,off,off,5th,0.831490,0.349532,0.270100,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
2,off,off,6th,0.838196,0.444443,0.343639,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
3,off,off,1st,0.968881,0.203448,0.146705,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
4,off,off,4th,0.595053,0.524738,0.423400,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night
...,...,...,...,...,...,...,...,...,...,...
619,off,off,5th,0.653152,0.188000,0.121083,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day
620,off,off,6th,0.829233,0.122814,0.073136,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day
621,off,off,1st,0.997337,0.003931,0.001014,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day
622,off,off,4th,0.665554,0.201290,0.134901,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day


# Modify Dataframes

In [151]:
def modify_df(work_df, off_df):
    '''Modify Dataframes columns and values to match/flow with specific questions
    
    Arguments:
    work_df -- Dataframe
    off_df -- Dataframe
    
    Return: tuple/list 
    work_df -- Dataframe
    off_df -- Dataframe
    '''
    # 0 is actually 11pm - 3am, ..., 5 is 7pm - 11pm
    # day starts with 2, which is 7am -11am, night starts with 5, which is 7pm - 11pm
    day_map = {'3rd': "11:00–15:00 h", '4th': "15:00–19:00 h", '5th': "19:00–23:00 h", '6th': "23:00–3:00 h", '1st': "3:00–7:00 h", '2nd': "7:00–11:00 h"}
    night_map = {'6th': '23:00–3:00 h', '1st': '3:00–7:00 h', '2nd': '7:00–11:00 h', '3rd': '11:00–15:00 h', '4th': '15:00–19:00 h', '5th': '19:00–23:00 h'}
    shift_map = {'day': 'day-shift', 'night': 'night-shift'}
    
    work_df_copy = work_df.copy()
    # work_df_copy = work_df_copy.drop(columns=["age",  "gender"])
    update_work_df = work_df_copy.replace({"time": day_map})
    update_work_df = update_work_df.replace({"shift": shift_map})
    
    off_df_copy = off_df.copy()
    update_off_df = off_df_copy.replace({"time": night_map})
    update_off_df = update_off_df.replace({"shift": shift_map})
                            
    return update_work_df, update_off_df

In [153]:
modified_work_df, modified_off_df = modify_df(work_df, off_df)

In [154]:
# modified_work_df

,Unnamed: 0,work,time,rest,step_ratio,run_ratio,id,age,gender,shift
0,work,work,11:00–15:00 h,0.870900,0.294175,0.236790,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night-shift
1,work,work,19:00–23:00 h,0.628179,0.492380,0.402107,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night-shift
2,work,work,23:00–3:00 h,0.901501,0.477633,0.377808,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night-shift
3,work,work,3:00–7:00 h,0.948894,0.374328,0.288105,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night-shift
4,work,work,15:00–19:00 h,0.764789,0.324393,0.271522,02581754-36cd-4b23-85ea-bf995c6dec83,< 40 Years,Male,night-shift
...,...,...,...,...,...,...,...,...,...,...
607,work,work,19:00–23:00 h,0.572392,0.268371,0.162666,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day-shift
608,work,work,23:00–3:00 h,0.788603,0.169805,0.098312,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day-shift
609,work,work,3:00–7:00 h,0.986683,0.003109,0.001196,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day-shift
610,work,work,15:00–19:00 h,0.610316,0.271353,0.172747,fcb14a4c-1ffa-4315-872a-a38833459885,>= 40 Years,Female,day-shift


# Load Generated Specific Questions

In [4]:
base = "/Users/brinkley97/Documents/development/lab-kcad/"
path_to_questions = "tiles-day-night/my_code"
name_of_questions = "generateSpecificQuestions.ipynb"
questions = base + path_to_questions + name_of_questions

In [5]:
# %load questions
/Users/brinkley97/Documents/development/lab-kcad/tiles-day-night/my_codegenerateSpecificQuestions.ipynb

In [6]:
%run "generateSpecificQuestions.ipynb"

In [11]:
# fig_2

# Py + R Code Integration

In [8]:
%load_ext rpy2.ipython

In [9]:
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages

# load and activate() bc if NOT error (Conversion 'py2rpy' not defined for objects of type '<class 'pandas.core.frame.DataFrame'>') will appear
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [10]:
report = rpackages.importr('report')

In [204]:
def figure_2(generated_specific_questions, work_df, off_df):
    '''Reproduce Table 2
    
    Arguments:
    generated_specific_questions -- list
    work_df -- Dataframe
    off_df -- Dataframe
    
    Return:
    
    '''
    # print(type(generated_specific_questions), generated_specific_questions)
    # print(type(work_df), work_df)
    # print(type(off_df), off_df)
    work_data_columns = list(work_df.keys())
    work_data_columns.append("shift")
    # print(work_data_columns)
    times = ['23:00–3:00 h', '3:00–7:00 h', '7:00–11:00 h', '11:00–15:00 h', '15:00–19:00 h', '19:00–23:00 h']
    # work_data_columns == time
    
    # work_data_columns = sum(work_data_columns, [])
    for time in times:
        work_data_columns.append(time)
    print(work_data_columns)
    
    
    for generated_specific_question in range(len(generated_specific_questions)):
        specific_question = generated_specific_questions[generated_specific_question]
        print("specific_question : ", specific_question)
        
        # word = specific_question.split("*")
        # print(word)
        
        for work_data_column in work_data_columns: 
            d = work_data_column
            # print(d)
            if work_data_column in specific_question.split("*"):
                print(work_data_column, True)
            else:
                print(work_data_column, False)

In [205]:
figure_2(fig_2, modified_work_df, modified_off_df)

['Unnamed: 0', 'work', 'time', 'rest', 'step_ratio', 'run_ratio', 'id', 'age', 'gender', 'shift', 'shift', '23:00–3:00 h', '3:00–7:00 h', '7:00–11:00 h', '11:00–15:00 h', '15:00–19:00 h', '19:00–23:00 h']
specific_question :  on both *work* day and *off* day, what is the effect of primarily *day-shift* nurses and primarily *night-shift* nurses on *mid* between the hours of *23:00–3:00 h* ?
Unnamed: 0 False
work True
time False
rest False
step_ratio False
run_ratio False
id False
age False
gender False
shift False
shift False
23:00–3:00 h True
3:00–7:00 h False
7:00–11:00 h False
11:00–15:00 h False
15:00–19:00 h False
19:00–23:00 h False
specific_question :  on both *work* day and *off* day, what is the effect of primarily *day-shift* nurses and primarily *night-shift* nurses on *mid* between the hours of *3:00–7:00 h* ?
Unnamed: 0 False
work True
time False
rest False
step_ratio False
run_ratio False
id False
age False
gender False
shift False
shift False
23:00–3:00 h False
3:00–7:00 